# 0. Justificación de uso de Spotify para sacar toptracks.

# 1. Sacar datos Spotify de 5 años, por países. 

In [1]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# Coloca tus credenciales aquí
CLIENT_ID = ''
CLIENT_SECRET = ''
# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

SpotifyOauthError: No client_id. Pass it or set a SPOTIPY_CLIENT_ID environment variable.

Aquí está el diccionario de países por el que hay que iterar. Sacaremos cada una 4 df, uno por cada grupo de países. 

In [3]:
paises_mediterraneos = {
    "ES": "España",
    "FR": "Francia",
    "MC": "Mónaco",
    "IT": "Italia",
    "SI": "Eslovenia",
    "HR": "Croacia",
    "BA": "Bosnia y Herzegovina",
    "ME": "Montenegro",
    "AL": "Albania",
    "GR": "Grecia",
    "MT": "Malta",
    "CY": "Chipre"}
paises_este_europa = {
    "BY": "Bielorrusia",
    "BG": "Bulgaria",
    "CZ": "Chequia",
    "SK": "Eslovaquia",
    "HU": "Hungría",
    "MD": "Moldavia",
    "PL": "Polonia",
    "RO": "Rumanía",
    "UA": "Ucrania"}
paises_norte_europa = {
    "DK": "Dinamarca",
    "EE": "Estonia",
    "FI": "Finlandia",
    "IS": "Islandia",
    "IE": "Irlanda",
    "LT": "Lituania",
    "LV": "Letonia",
    "NO": "Noruega",
    "SE": "Suecia",
    "GB": "Reino Unido"}
paises_europa_central = {
    "AT": "Austria",
    "DE": "Alemania",
    "CH": "Suiza",
    "LI": "Liechtenstein",
    "BE": "Bélgica",
    "LU": "Luxemburgo",
    "NL": "Países Bajos"}

Esta función sirve para ir sacando los datos para cada grupo de países. Habrá que hacer una llamada para cada grupo de países (cada diccionario de los de arriba). 

La función toma como argumentos el diccionario y el rango de años que nos interese en el formato `2000-2004`

In [4]:
def extraccion_por_paises(diccionario_zonas, años):
    todos_los_datos = [] # Esto es una lista de dicts donde se van guardando los diccionarios de cada país
    for pais in diccionario_zonas.keys():
        datos_rango = sp.search(q = f'year:{años}', limit=50, offset=0, type='track', market={pais}) # Cada una tiene que cambiar en q el rango de años
        datos_por_pais = {}
        datos_por_pais['pais'] = pais
        datos_por_pais['datos'] = datos_rango
        todos_los_datos.append(datos_por_pais)
    return todos_los_datos
   

In [5]:
mediterraneo = extraccion_por_paises(paises_mediterraneos, "2005-2009")
este = extraccion_por_paises(paises_este_europa, "2005-2009")
norte = extraccion_por_paises(paises_norte_europa, "2005-2009")
centro = extraccion_por_paises(paises_europa_central, "2005-2009")

## 1.1. Filtrar ahora los datos que hemos sacado

En el diccionario `prueba` es donde iremos metiendo los datos que extraigamos

In [6]:
datos_mediterraneo = {'nombre': [], 'popularity': [], 'artista': [], 'pais': [], 'genero': []}
datos_este = {'nombre': [], 'popularity': [], 'artista': [], 'pais': [], 'genero': []}
datos_norte = {'nombre': [], 'popularity': [], 'artista': [], 'pais': [], 'genero': []}
datos_centro = {'nombre': [], 'popularity': [], 'artista': [], 'pais': [], 'genero': []}

Con el código de abajo se extraen los datos de spotify. Luego hacemos una función. 

La función `filtro_de_datos` toma como argumento la salida de la función `extraccion_por_paises`, que es una lista donde hemos ido almacenando las salidas de las llamadas. 

In [7]:
def filtro_de_datos(lista_datos_zonas, datos):
    for datos_pais in lista_datos_zonas:
        for element in datos_pais['datos']['tracks']['items']:
            datos['pais'].append(datos_pais['pais'])
            datos['popularity'].append(element['popularity'])
            datos['nombre'].append(element['name'])
            artistas = []
            for artista in element['artists']:
                artistas.append(artista['name'])   
            artistas_str = ', '.join(artistas)
            datos['artista'].append(artistas_str)

In [8]:
lista_dicts = [datos_mediterraneo, datos_este, datos_norte, datos_centro]
lista_resultados = [mediterraneo, este, norte, centro]

for i in range(len(lista_dicts)):
    filtro_de_datos(lista_resultados[i],lista_dicts[i])

# 2. Sacar de Lastfm el género de las canciones. 

In [9]:
import requests

In [ ]:
# Coloca tus credenciales de lastfm aquí
CLIENT_ID = ''
API_KEY = ''

last_fm_connect = requests.get(f"http://www.last.fm/api/auth/?api_key={API_KEY}")

Aquí hay algunas pruebas que hemos hecho para ver cómo sacar el género de una canción con la API de lastfm.

In [11]:

test_2 = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist=Sasha&track=here+she+comes+again&api_key={API_KEY}&autocorrect=1&format=json")

In [12]:
test_2.json()['toptags']['tag'][0]['name'] # Nos quedaríamos con la primera porque es la que más se repite (count más alto)

'pop'

🧠Recordatorio: Como la llamada test_2 nos devuelve varios géneros (tags), decidimos quedarnos con el primero, que es el más utilizado. 

Queremos extraer el género para introducirlo en el diccionario `prueba`, por lo que hacemos dos listas, una con los nombres de las canciones y otra con los artistas. Iremos iterando por las dos a la vez para poder hacer la llamada a la API de lastfm.

In [11]:
lista_nombres_mediterraneo = list(datos_mediterraneo.values())[0]
lista_artistas_mediterraneo = list(datos_mediterraneo.values())[2]

lista_nombres_este = list(datos_este.values())[0]
lista_artistas_este = list(datos_este.values())[2]

lista_nombres_norte = list(datos_norte.values())[0]
lista_artistas_norte = list(datos_norte.values())[2]

lista_nombres_centro = list(datos_centro.values())[0]
lista_artistas_centro = list(datos_centro.values())[2]

Con este bucle hacemos la llamada a la API. El try/except controla que el código no se pare cuando haya alguna llamada que nos de un error o nos devuelva una canción sin tags. 

In [30]:
def llamada_lastfm(lista_nombres, lista_artistas,datos):
    for i in range(len(lista_nombres)):
        try:
            nombre = str(lista_nombres[i]).replace(' ', '+')
            artista = lista_artistas[i].replace(' ', '+').replace(',', '+&')
            llamada_genero = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist={artista}&track={nombre}&api_key={API_KEY}&autocorrect=1&format=json")
            try:
                genero_cancion = llamada_genero.json()['toptags']['tag'][0]['name']
                datos['genero'].append(genero_cancion)
            except KeyError:
                datos['genero'].append("NULL")
                print(f"La canción {lista_nombres[i]} no está registrada en lastfm")
                
            except IndexError:
                datos['genero'].append("NULL")
                print(f"La canción {lista_nombres[i]} no tiene tags")
            
        except:
            datos['genero'].append("NULL")
            print(f"Hay problemas con la canción {lista_nombres[i]}")
            
          

In [13]:
llamada_lastfm(lista_nombres_mediterraneo, lista_artistas_mediterraneo, datos_mediterraneo)

La canción Colillas en el sueño no tiene tags
La canción Me equivocaría otra vez no tiene tags
La canción Y pasa el tiempo no tiene tags
La canción Tu Vuoi Da Me Qualcosa - Live no está registrada en lastfm
La canción Clapton: Tears in Heaven/video no tiene tags
La canción Caro diario no tiene tags
La canción J'étais comme eux no tiene tags
La canción Stars of CCTV - Live at the Astoria no está registrada en lastfm
La canción Sé lo que hicisteis - 2006 Remastered Version no tiene tags
La canción Sí, sí - Directo 07 no está registrada en lastfm
La canción Save Me no está registrada en lastfm
La canción Hips Don't Lie (feat. Wyclef Jean) no está registrada en lastfm
La canción Con Tanto Héroe - Live no está registrada en lastfm
La canción Ella Y Yo - Featuring Don Omar no está registrada en lastfm
La canción Donde Estan las Llaves no tiene tags
Hay problemas con la cación Blue Man Group - Rock Concert Movement #10
La canción Smiley Faces - Live T4 Performance no tiene tags
La canción Cau

In [22]:
llamada_lastfm(lista_nombres_este, lista_artistas_este, datos_este)

La canción Tu Vuoi Da Me Qualcosa - Live no está registrada en lastfm
La canción Save Me no está registrada en lastfm
La canción Esqueça Que Eu Te Amo - Ao Vivo no está registrada en lastfm
La canción Tá No Meu Coração - Ao Vivo no está registrada en lastfm
La canción Just Dance no está registrada en lastfm
La canción Как на войне no tiene tags
La canción Disco Eterno - Gira Me Verás Volver no está registrada en lastfm
La canción Nossa História - Ao Vivo no está registrada en lastfm
La canción Dança Do Bole, Bole - Live no está registrada en lastfm
La canción Samba Menina / Ay Cosita Linda - Ao Vivo no está registrada en lastfm
La canción É Para Ficar - Live from Ilha de Magia, Florianópolis, Brazil/2008 no está registrada en lastfm
La canción Smack That no está registrada en lastfm
La canción Yeah! - T4 Performance no está registrada en lastfm
La canción Ainda Ontem Chorei De Saudade - Live From Villa Country, São Paulo, Brazil/2007 no está registrada en lastfm
La canción The Way I Ar

In [26]:
llamada_lastfm(lista_nombres_norte, lista_artistas_norte, datos_norte)

La canción Clapton: Tears in Heaven/video no tiene tags
La canción Save Me no está registrada en lastfm
La canción Da Rockwilder no está registrada en lastfm
La canción The Way I Are no está registrada en lastfm
La canción Tu Vuoi Da Me Qualcosa - Live no está registrada en lastfm
La canción Flashing Lights no tiene tags
La canción Esqueça Que Eu Te Amo - Ao Vivo no está registrada en lastfm
La canción Colillas en el sueño no tiene tags
La canción Homecoming no tiene tags
La canción Stars of CCTV - Live at the Astoria no está registrada en lastfm
La canción Hips Don't Lie (feat. Wyclef Jean) no está registrada en lastfm
La canción Just Dance no está registrada en lastfm
La canción J'étais comme eux no tiene tags
La canción Umbrella no está registrada en lastfm
La canción Tá No Meu Coração - Ao Vivo no está registrada en lastfm
La canción Start Of Something New no está registrada en lastfm
La canción Promiscuous no está registrada en lastfm
La canción Boing! no está registrada en lastfm

In [27]:
llamada_lastfm(lista_nombres_centro, lista_artistas_centro, datos_centro)

La canción Tu Vuoi Da Me Qualcosa - Live no está registrada en lastfm
La canción Clapton: Tears in Heaven/video no tiene tags
La canción Just Dance no está registrada en lastfm
La canción Tattoo (feat. Fabo) - D4L no está registrada en lastfm
La canción Save Me no está registrada en lastfm
La canción Stars of CCTV - Live at the Astoria no está registrada en lastfm
La canción The Way I Are no está registrada en lastfm
La canción The Call - Live in Ireland, 2009 no está registrada en lastfm
La canción Europe '03 Tour, LPTV Episode 4 no está registrada en lastfm
La canción Smiley Faces - Live T4 Performance no tiene tags
La canción Hips Don't Lie (feat. Wyclef Jean) no está registrada en lastfm
La canción Never Give Up no tiene tags
La canción Esqueça Que Eu Te Amo - Ao Vivo no está registrada en lastfm
La canción Colillas en el sueño no tiene tags
La canción Promiscuous no está registrada en lastfm
La canción J'étais comme eux no tiene tags
La canción Start Of Something New no está regis

In [29]:
len(datos_norte['genero']) # Aquí podemos mirar luego un DISTINCT de SQL para ver qué pasa con las tags estas del demoño

500

Ahora que ya tenemos todo el diccionario `prueba` completo, lo convertimos a Data Frame.

In [17]:
import pandas as pd

In [18]:
df_mediterraneo = pd.DataFrame(datos_mediterraneo)

In [31]:
df_este = pd.DataFrame(datos_este)
df_norte = pd.DataFrame(datos_norte)
df_centro = pd.DataFrame(datos_centro)

In [19]:
df_mediterraneo.index.name="id"

In [32]:
df_este.index.name = "id"
df_centro.index.name = "id"
df_norte.index.name = "id"

Por último, pasamos el Data Frame a csv. 

🧠 Recuerda cambiar el nombre del csv antes de ejecutar la celda.

In [21]:
df_mediterraneo.to_csv("mediterraneo_2005_2009.csv")

In [ ]:
df_este.to_csv("este_2005_2009.csv")

In [34]:
df_centro.to_csv("centro_2005_2009.csv")

In [35]:
df_norte.to_csv("norte_2005_2009.csv")

# 3. Crear la bbdd

# 4. Alimentar la bbdd

# 5. Next steps

- Sacar más de 50 canciones por llamada